# **Lab 2 Basim Sherief 1210207**


### **Requirement 2**

1) Complete the provided matrix addition example, following these cases:
        
        A.   kernel1: each thread produces one output matrix element
        B.   kernel2: each thread produces one output matrix row
        C.   kernel3: each thread produces one output matrix column
  Analyze the pros and cons of each of the kernels above by using nvprof with large matrix sizes to validate your posize_ts. Collect your insights in a PDF report and explain them.


In [3]:
# Setup cuda environment
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\basim\appdata\local\temp\pip-req-build-w96ql_t7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.
Source files will be saved in "C:\Users\basim\AppData\Local\Temp\tmpxkuu53mo".


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\basim\AppData\Local\Temp\pip-req-build-w96ql_t7'


**Generate Testcases**

In [ ]:
import numpy as np

def generate_matrix_test_cases(num_tests, min_dim, max_dim, min_val, max_val, output_file):
    """
    Generate test cases for matrix addition
    Parameters:
    - num_tests: number of test cases
    - min_dim: minimum dimension (rows/cols)
    - max_dim: maximum dimension (rows/cols)
    - min_val: minimum value in matrices
    - max_val: maximum value in matrices
    - output_file: path to output file
    """
    with open(output_file, 'w') as f:
        # Write number of test cases
        f.write(f"{num_tests}\n")
        
        for _ in range(num_tests):
            # Generate random dimensions
            rows = np.random.randint(4096, 4096 + 1)
            cols = np.random.randint(256, 256 + 1)
            
            # Write dimensions
            f.write(f"{rows} {cols}\n")
            
            # Generate and write first matrix
            matrix1 = np.random.uniform(min_val, max_val, (rows, cols))
            for row in matrix1:
                f.write(" ".join(f"{x:.3f}" for x in row) + "\n")
            
            # Generate and write second matrix
            matrix2 = np.random.uniform(min_val, max_val, (rows, cols))
            for row in matrix2:
                f.write(" ".join(f"{x:.3f}" for x in row) + "\n")

# Set your parameters here
params = {
    'num_tests': 1,              # Number of test cases
    'min_dim': 1024,               # Minimum matrix dimension
    'max_dim': 1024,               # Maximum matrix dimension
    'min_val': -50000000.0,           # Minimum value in matrices
    'max_val': 5000000000.0,            # Maximum value in matrices
    'output_file': './Input_TestCases/inputfile.txt'  # Output file name
}

# Run the generator with the specified parameters
if __name__ == "__main__":
    generate_matrix_test_cases(**params)

# **CPU Only**
# Vector addition in pure C (CPU-only execution)

In [5]:
%%writefile kernel0.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <time.h>

#define MAX_ERR 1e-6

// Function to perform vector addition
void vector_add(double *out, double *a, double *b, size_t  n) {
    for (size_t  i = 0; i < n; i++) {
        out[i] = a[i] + b[i];
    }
}

int main(int argc, char* argv[]) {
    FILE *file_reading;
    int numberOfTests;
    size_t rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }

    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu",&rows, &cols);
    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    if (A == NULL || B == NULL || C == NULL) {
        printf("Memory allocation failed!\n");
        fclose(file_reading);
        return 1;
    }

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }



    // Start timing
    clock_t start = clock();

    // Perform vector addition
    vector_add(C, A, B, rows * cols);

    // End timing
    clock_t end = clock();

    // Calculate the elapsed time in seconds
    double time_spent = (double)(end - start) / CLOCKS_PER_SEC * 1000.0;

    printf("Time elapsed: %f ms\n", time_spent);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

  
    // Write results to output file
   // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
              // printf("%.3lf ", C[i * cols + j]);    
            fprintf(file_writing, "%.3lf ", C[i * cols + j]); // Write double with 2 decimal places
        }
         // printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Free allocated memory
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel0.cu


In [6]:
# Compile the CUDA program
!nvcc kernel0.cu -o kernel0.exe


kernel0.cu
tmpxft_0000514c_00000000-10_kernel0.cudafe1.cpp
   Creating library kernel0.lib and object kernel0.exp


In [37]:
# Run the executable (Windows style)
!.\kernel0.exe inputfile.txt outputfile_cpu.txt

Time elapsed: 0.000000 ms
Vector addition completed successfully!
Time elapsed: 0.000000 ms
Vector addition completed successfully!


In [8]:

# For profiling with nvprof (Windows style)
!nvprof .\kernel0.exe inputfile.txt outputfile_cpu.txt

# kernel1: each thread produces one output matrix element


In [23]:
%%writefile kernel1.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t row = blockIdx.y * blockDim.y + threadIdx.y;
    size_t col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < rows && col < cols) {
        size_t idx = row * cols + col;
        C[idx] = A[idx] + B[idx];
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
              // printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
         // printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel1.cu


In [24]:
# Compile the CUDA program
!nvcc kernel1.cu -o kernel1.exe

# Run the executable (Windows style)
!.\kernel1.exe inputfile.txt outputfile_kernel1.txt

# perfomance for cuda is depricacted but can be used in kaggle notebook
# !nvprof .\kernel1.exe inputfile.txt outputfile_cpu.txt



kernel1.cu
tmpxft_00001a4c_00000000-10_kernel1.cudafe1.cpp
   Creating library kernel1.lib and object kernel1.exp
Vector addition completed successfully!
Vector addition completed successfully!


In [25]:
# For profiling with nvprof (Windows style) the new one  for system-level profiling
# !nsys profile --stats=true .\kernel1.exe inputfile.txt outputfile_cpu.txt
# # , for kernel details using Nsight Compute:
# !ncu --set full .\kernel1 inputfile.txt outputfile_cpu.txt
!ncu --metrics gpu__time_duration .\kernel1.exe inputfile.txt outputfile_cpu.txt

==PROF== Connected to process 16096 (E:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Lab_2_Notebook_Solution\kernel1.exe)
==PROF== Profiling "matrixAddKernel1" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "matrixAddKernel1" - 1: 0%....50%....100% - 1 pass
Vector addition completed successfully!
Vector addition completed successfully!
==PROF== Disconnected from process 16096
[16096] kernel1.exe@127.0.0.1
  matrixAddKernel1(double *, double *, double *, unsigned long long, unsigned long long) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: Command line profiler metrics
    ---------------------- ----------- ------------
    Metric Name            Metric Unit Metric Value
    ---------------------- ----------- ------------
    gpu__time_duration.avg          us         2.62
    gpu__time_duration.max          us         2.62
    gpu__time_duration.min          us         2.62
    gpu__time_duration.sum          us         2.62
    

# kernel2: each thread produces one output matrix row


In [3]:
%%writefile ./cude_kernels/kernel2.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < rows) {
        for(size_t col =0 ; col < cols; col++){
              size_t idx = row * cols + col;
            C[idx] = A[idx] + B[idx];
        }
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "a"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
            //  printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
        // printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Writing ./cude_kernels/kernel2.cu


In [5]:
!nvcc ./cude_kernels/kernel2.cu -o cude_kernels/kernel2.exe
!nvprof ./cude_kernels/kernel2.exe inputfile.txt outputfile_kernel2.txt

kernel2.cu
tmpxft_00002160_00000000-10_kernel2.cudafe1.cpp
   Creating library cude_kernels\kernel2.lib and object cude_kernels\kernel2.exp


In [6]:
# For profiling with nvprof (Windows style) the new one  for system-level profiling
# !nsys profile --stats=true .\kernel2.exe inputfile.txt outputfile_cpu.txt
# # , for kernel details using Nsight Compute:
# !ncu --set full .\kernel2 inputfile.txt outputfile_cpu.txt
!ncu --metrics gpu__time_duration .\cude_kernels\kernel2.exe inputfile.txt outputfile_kernel2.txt

==PROF== Connected to process 12396 (E:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Lab_2_Notebook_Solution\cude_kernels\kernel2.exe)
==PROF== Profiling "matrixAddKernel1" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "matrixAddKernel1" - 1: 0%....50%....100% - 1 pass
Vector addition completed successfully!
Vector addition completed successfully!
==PROF== Disconnected from process 12396
[12396] kernel2.exe@127.0.0.1
  matrixAddKernel1(double *, double *, double *, unsigned long long, unsigned long long) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: Command line profiler metrics
    ---------------------- ----------- ------------
    Metric Name            Metric Unit Metric Value
    ---------------------- ----------- ------------
    gpu__time_duration.avg          us         3.33
    gpu__time_duration.max          us         3.33
    gpu__time_duration.min          us         3.33
    gpu__time_duration.sum          us     

# kernel3: each thread produces one output matrix col


In [35]:
%%writefile kernel3.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t col = blockIdx.x * blockDim.x + threadIdx.x;
    if ( col < cols) {
        for(size_t row = 0; row< rows ; row++){
            size_t idx = row * cols + col;
            C[idx] = A[idx] + B[idx];
        }
   
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "a"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
           // printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
       // printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel3.cu


In [36]:
!nvcc kernel3.cu -o kernel3.exe
!nvprof ./kernel3 inputfile.txt outputfile_kernel3.txt

kernel3.cu
tmpxft_0000511c_00000000-10_kernel3.cudafe1.cpp
   Creating library kernel3.lib and object kernel3.exp


In [31]:
# For profiling with nvprof (Windows style) the new one  for system-level profiling
# !nsys profile --stats=true .\kernel3.exe inputfile.txt outputfile_cpu.txt
# # , for kernel details using Nsight Compute:
# !ncu --set full .\kernel3 inputfile.txt outputfile_cpu.txt
!ncu --metrics gpu__time_duration .\kernel3.exe inputfile.txt outputfile_kernel3.txt

==PROF== Connected to process 20144 (E:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Lab_2_Notebook_Solution\kernel3.exe)
==PROF== Profiling "matrixAddKernel1" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "matrixAddKernel1" - 1: 0%....50%....100% - 1 pass
Vector addition completed successfully!
Vector addition completed successfully!
==PROF== Disconnected from process 20144
[20144] kernel3.exe@127.0.0.1
  matrixAddKernel1(double *, double *, double *, unsigned long long, unsigned long long) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: Command line profiler metrics
    ---------------------- ----------- ------------
    Metric Name            Metric Unit Metric Value
    ---------------------- ----------- ------------
    gpu__time_duration.avg          us         3.26
    gpu__time_duration.max          us         3.26
    gpu__time_duration.min          us         3.26
    gpu__time_duration.sum          us         3.26
    

2. It is required to sum all elements in a 3d volume as follows:
- First kernel will sum all elements across the z-dimension and produces a 2d matrix
- Second kernel will reduce the 2d matrix into a 1d vector
- The resulted vector should be reduced int a single element using the CPU
- Print "Kernel {kernel name}" Started after calling each kernel, what do you notice?

example input and output files for the second requirement are attached

your code should take two arguments which are the input and output files (you may use c++)

In [32]:
%%writefile kernel.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel to sum across the z-dimension
__global__ void sumZDimension(double *input, double *output, size_t width, size_t height, size_t depth) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        double sum = 0.0;
        for (size_t z = 0; z < depth; ++z) {
            sum += input[(z * height + y) * width + x];
        }
        output[y * width + x] = sum;
    }
}

// Kernel to reduce 2D matrix to 1D vector
__global__ void reduce2DTo1D(double *input, double *output, size_t width, size_t height) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;

    if (x < width) {
        double sum = 0.0;
        for (size_t y = 0; y < height; ++y) {
            sum += input[y * width + x];
        }
        output[x] = sum;
    }
}

// Function to reduce 1D vector to a single element on the CPU
double reduce1DToSingle(double *input, size_t length) {
    double sum = 0.0;
    for (size_t i = 0; i < length; ++i) {
        sum += input[i];
    }
    return sum;
}

// Function to read input data from file
void readInputFile(const char *filename, double **data, size_t *width, size_t *height, size_t *depth) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    fscanf(file, "%zu %zu %zu", width, height, depth);

    size_t size = (*width) * (*height) * (*depth);
    *data = (double *)malloc(size * sizeof(double));

    for (size_t i = 0; i < size; ++i) {
        fscanf(file, "%lf", &(*data)[i]);
    }

    fclose(file);
}

// Function to write output data to file
void writeOutputFile(const char *filename, double result) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Round to 3 decimal places
    result = round(result * 1000.0) / 1000.0;

    fprintf(file, "%.3f\n", result);
    fclose(file);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <inputfile> <outputfile>\n", argv[0]);
        return -1;
    }

    const char *inputFile = argv[1];
    const char *outputFile = argv[2];

    // Load input data from file
    double *h_input;
    size_t width, height, depth;
    readInputFile(inputFile, &h_input, &width, &height, &depth);

    // Allocate memory for input and output
    double *d_input, *d_output2D, *d_output1D;
    cudaMalloc(&d_input, width * height * depth * sizeof(double));
    cudaMalloc(&d_output2D, width * height * sizeof(double));
    cudaMalloc(&d_output1D, width * sizeof(double));

    // Copy input data to device
    cudaMemcpy(d_input, h_input, width * height * depth * sizeof(double), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch first kernel
    printf("Kernel sumZDimension Started\n");
    sumZDimension<<<gridSize, blockSize>>>(d_input, d_output2D, width, height, depth);
    cudaDeviceSynchronize();

    // Launch second kernel
    printf("Kernel reduce2DTo1D Started\n");
    reduce2DTo1D<<<(width + blockSize.x - 1) / blockSize.x, blockSize.x>>>(d_output2D, d_output1D, width, height);
    cudaDeviceSynchronize();

    // Copy result back to host
    double *h_output1D = (double *)malloc(width * sizeof(double));
    cudaMemcpy(h_output1D, d_output1D, width * sizeof(double), cudaMemcpyDeviceToHost);

    // Reduce 1D vector to a single element on the CPU
    double result = reduce1DToSingle(h_output1D, width);

    // Save result to output file
    writeOutputFile(outputFile, result);

    // Free memory
    cudaFree(d_input);
    cudaFree(d_output2D);
    cudaFree(d_output1D);
    free(h_input);
    free(h_output1D);

    return 0;
}

Overwriting kernel.cu


In [33]:
# Compile the CUDA program
!nvcc kernel.cu -o kernel.exe

# Run the executable (Windows style)
!.\kernel.exe inputfile.txt outputfile_cpu.txt

# perfomance for cuda is depricacted but can be used in kaggle notebook
# !nvprof .\kernel.exe inputfile.txt outputfile_cpu.txt


kernel.cu
tmpxft_00000cc0_00000000-10_kernel.cudafe1.cpp
   Creating library kernel.lib and object kernel.exp
Kernel sumZDimension Started
Kernel reduce2DTo1D Started


In [34]:
# For profiling with nvprof (Windows style) the new one  for system-level profiling
# !nsys profile --stats=true .\kernel0.exe inputfile.txt outputfile_cpu.txt
# # , for kernel details using Nsight Compute:
# !ncu --set full .\kernel0 inputfile.txt outputfile_cpu.txt
!ncu --metrics gpu__time_duration .\kernel.exe inputfile.txt outputfile_cpu.txt

==PROF== Connected to process 3444 (E:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Lab_2_Notebook_Solution\kernel.exe)
==PROF== Profiling "sumZDimension" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "reduce2DTo1D" - 1: 0%....50%....100% - 1 pass
Kernel sumZDimension Started
Kernel reduce2DTo1D Started
==PROF== Disconnected from process 3444
[3444] kernel.exe@127.0.0.1
  sumZDimension(double *, double *, unsigned long long, unsigned long long, unsigned long long) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: Command line profiler metrics
    ---------------------- ----------- ------------
    Metric Name            Metric Unit Metric Value
    ---------------------- ----------- ------------
    gpu__time_duration.avg          us         3.30
    gpu__time_duration.max          us         3.30
    gpu__time_duration.min          us         3.30
    gpu__time_duration.sum          us         3.30
    ---------------------- -----